In [3]:
from requests import get, post
import pandas as pd
import numpy as np
import sympy as sp
from matplotlib import pyplot as plt

# Carga de Dados
**Origem dos dados**

[Binance](https://www.binance.com)

[Documentação da API](https://github.com/binance/binance-spot-api-docs/blob/master/rest-api.md#klinecandlestick-data)

In [4]:
# Endpoints
url = 'https://api3.binance.com'
klines = url + '/api/v3/klines'
info = url + '/api/v3/exchangeInfo'

In [102]:
# Colunas
col = [
    'Open time','Open','High','Low','Close','Volume','Close time',
    'Quote asset volume','Number of trades','Taker buy base asset volume',
    'Taker buy quote asset volume','Ignore'
]

# Parametros
params = {
    "btc": {
        "symbol": "BTCUSDT",
        "interval": "1d",
        "limit": 700
    },
    "eth": { 
        "symbol": "ETHUSDT",
        "interval": "1d",
        "limit": 700
    },
    "ltc": {
        "symbol": "ETHUSDT",
        "interval": "1d",
        "limit": 700
    },
    "bnb": {
        "symbol": "BNBUSDT",
        "interval": "1d",
        "limit": 700
    },
    "ada": {
        "symbol": "ADAUSDT",
        "interval": "1d",
        "limit": 700
    },
    "xmr": {
        "symbol": "XMRUSDT",
        "interval": "1d",
        "limit": 700
    },
    "eos": {
        "symbol": "EOSUSDT",
        "interval": "1d",
        "limit": 700
    },
    "dash": {
        "symbol": "DASHUSDT",
        "interval": "1d",
        "limit": 700
    },
    "nano": {
        "symbol": "NANOUSDT",
        "interval": "1d",
        "limit": 700
    },
    "tron": {
        "symbol": "TRXUSDT",
        "interval": "1d",
        "limit": 700
    },
    "zcash": {
        "symbol": "ZECUSDT",
        "interval": "1d",
        "limit": 700
    },
    "waves": {
        "symbol": "WAVESUSDT",
        "interval": "1d",
        "limit": 700
    }
}

In [103]:
def clean(x):    
    # Obtendo dados
    asset = get(klines, params=params[x]).json()

    # Limando - > deixando apenas as colunas: (close, time_close)
    asset = np.array(asset, dtype=float)[..., 4]
    
    return asset

In [110]:
df = pd.DataFrame(np.array([*map(clean, params.keys())]).T, columns=params.keys())

In [111]:
# setar index
get(klines, params=params['btc']).json()

,btc,eth,ltc,bnb,ada,xmr,eos,dash,nano,tron,zcash,waves
0,5236.90,178.72,178.72,17.9777,0.08604,70.47,5.5003,135.30,1.5876,0.03033,72.19,2.8349
1,5150.00,174.60,174.60,18.2984,0.08288,67.75,5.5231,131.40,1.6745,0.02960,70.30,2.7930
2,5308.25,177.12,177.12,18.2400,0.08947,70.86,5.8417,129.74,1.6131,0.02997,71.75,2.7788
3,5017.37,164.07,164.07,17.5555,0.08297,65.97,5.2778,122.30,1.4550,0.02641,69.61,2.6353
4,5048.01,163.19,163.19,18.3333,0.08250,65.73,5.3313,118.93,1.6300,0.02628,70.94,2.6425
...,...,...,...,...,...,...,...,...,...,...,...,...
695,50349.37,1567.69,1567.69,240.5701,1.21311,218.70,3.8258,222.59,5.4039,0.04883,126.69,9.8182
696,48374.09,1539.23,1539.23,229.6369,1.11239,217.52,3.7483,211.18,5.1283,0.05142,122.03,9.4472
697,48751.71,1528.31,1528.31,225.4714,1.15908,205.84,3.6904,205.70,5.1502,0.05019,121.12,9.9125
698,48882.20,1650.35,1650.35,226.3814,1.12900,204.35,3.7207,208.61,5.1160,0.05015,125.19,9.9916


# Análise dos ativos
## Calculando Taxa de Retorno Simples, Logarítmicos e Volatilidade

**Retorno Simples:**
$$
r = \frac{P_i - P_f}{P_i} = \frac{P_f}{P_i} - 1
$$

**Retornos Logarítmicos**

$$
r = log(\frac{P_t}{P_{t-1}})
$$

In [113]:
def simple_return(asset):
    new_asset = (df[asset] / df[asset].shift(1)) - 1
    return new_asset

def log_return(asset):
    new_asset = (df[asset] / df[asset].shiftt(1)) - 1
    return new_asset

In [125]:
returns = pd.DataFrame([*map(simple_return, df)]).T.iloc[1:,:]

In [126]:
returns

,btc,eth,ltc,bnb,ada,xmr,eos,dash,nano,tron,zcash,waves
1,-0.016594,-0.023053,-0.023053,0.017839,-0.036727,-0.038598,0.004145,-0.028825,0.054737,-0.024069,-0.026181,-0.014780
2,0.030728,0.014433,0.014433,-0.003192,0.079513,0.045904,0.057685,-0.012633,-0.036668,0.012500,0.020626,-0.005084
3,-0.054798,-0.073679,-0.073679,-0.037527,-0.072650,-0.069009,-0.096530,-0.057345,-0.098010,-0.118785,-0.029826,-0.051641
4,0.006107,-0.005364,-0.005364,0.044305,-0.005665,-0.003638,0.010137,-0.027555,0.120275,-0.004922,0.019106,0.002732
5,-0.000553,-0.001226,-0.001226,0.015153,0.004121,-0.012323,-0.008553,0.001429,-0.052025,-0.009132,-0.026219,-0.000530
...,...,...,...,...,...,...,...,...,...,...,...,...
695,0.039403,0.054420,0.054420,0.003697,-0.007924,-0.006090,0.042168,0.030318,0.064137,0.041818,0.051282,0.042548
696,-0.039231,-0.018154,-0.018154,-0.045447,-0.083026,-0.005396,-0.020257,-0.051260,-0.051000,0.053041,-0.036783,-0.037787
697,0.007806,-0.007094,-0.007094,-0.018140,0.041973,-0.053696,-0.015447,-0.025949,0.004270,-0.023921,-0.007457,0.049253
698,0.002677,0.079853,0.079853,0.004036,-0.025952,-0.007239,0.008210,0.014147,-0.006641,-0.000797,0.033603,0.007980


In [12]:
def f(asset):
    # media diaria
    x = df[asset].mean() * 100
    
    # media anual
    y = df[asset].mean() * 365 * 100
    
    # variacao diaria 
    z = df[asset].std() * 100

    # desvio padrao anual
    w = df[asset].std() * 365 ** 0.5 * 100
    return [x,y,z,w]

col = ['Retorno Médio Diário', 'Retorno Médio Anual', 'Desvio Padrão Diário', 'Desvio Padrão Anual']

df_return = pd.DataFrame(map(f, returns), index=returns, columns=col)
df_return.style.format('{:.2f}%')

,Retorno Médio Diário,Retorno Médio Anual,Desvio Padrão Diário,Desvio Padrão Anual
btc_return,0.20%,74.48%,3.94%,75.18%
eth_return,0.25%,92.27%,5.03%,96.15%


## Volatilidade

**Variância**

```
Mede a distância entre os valores e a média.
var(x) = média dos  quadrados - quadrado da média
```
$$
var(x) = \frac{1}{n}\sum_{i=1}^{n}(x_i - \bar{x})^2
$$

$$
= \frac{1}{n}\sum_{i=1}^{n}x_i² - \bar{x}²
$$

**Desvio Padrão**
```
Mede a variação média
dp(x) = raiz quadrada da variância
```
$$
\sigma(x) = \sqrt{var(x)}
$$

**Covariância**

```
Mede o grau de interdependência entre as variáveis
Cov(x,y) = média dos produtos - produto das médias
cov(x,y) >  0: x e y se movem na mesma direção
cov(x,y) <  0: x e y se movem em direções contrárias
cov(x,y) == 0: x e y são independentes
```
$$
cov(x, y) = \frac{1}{n}\sum_{i=1}^{n}(x_i - \bar{x})\cdot(y_i - \bar{y})
$$
$$
= \frac{1}{n}\sum_{i=1}^{n}x_i^2-\bar{x}^2
$$

**Coeficiente de Correlação**
```
Ajusta a cov(x,y) para um intervalo de [-1;1]
Corr(x,y) = {
        média dos produtos - produto das médias
        roduto das raiz quadrada da variância de x por y
}        
corr(x,y) = +ou- 1: o movimento de uma variável explica completamente o movimento da outra
corr(x,y) =  1: correlação positiva perfeita entre as duas variáveis.
corr(x,y) = -1: correlação negativa perfeita entre as duas variáveis.
corr(x,y) =  0: correlação nula
```

$$
corr(x,y) = \frac{cov(x,y)}{\sigma(x)\cdot \sigma(y)}
$$

**Covariância e Correlação**
$$
\begin{bmatrix}
var(x) && cov(x,y) \\
cov(x,y) && var(y)
\end{bmatrix}
$$

In [11]:
def f(asset):
    # variancia diaria
    x = df[asset].var()
    
    # variancia anual
    y = df[asset].var() * 250
    
    return [x,y]

col = ['Variância Média Diário', 'Variância Média Anual']

df_return = pd.DataFrame(map(f, returns), index=returns, columns=col)

In [12]:
df_return

,Variância Média Diário,Variância Média Anual
btc_return,0.001548,0.387123
eth_return,0.002533,0.633203


In [13]:
def g(df, col):
    # covariancia diaria
    cov = df[col].cov()
    
    # covariancia anual
    cov_Y = df[col].cov() * 250
    
    # correlacao
    corr = df[col].corr()

    return {
        'covariancia': cov,
        'covariancia_Y':cov_Y,
        'correlacao':corr
    }

In [14]:
cov_corr = g(df, ['btc_return', 'eth_return'])

In [15]:
cov_corr['covariancia']

,btc_return,eth_return
btc_return,0.001548,0.001610
eth_return,0.001610,0.002533


In [16]:
cov_corr['covariancia_Y']

,btc_return,eth_return
btc_return,0.387123,0.402613
eth_return,0.402613,0.633203


In [17]:
cov_corr['correlacao']

,btc_return,eth_return
btc_return,1.000000,0.813188
eth_return,0.813188,1.000000


# Risco de um Portifólio Simples (2 ativos)

$$
(w_1\sigma_1 + w_2\sigma_2)^2 => w_1 + w_2 = 1
$$